In [1]:
# import opensim as osim
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from OA_utils.data_utils import *

In [2]:
data_dir = "C:\\Users\\bakel\\Desktop\\GRFMuscleModel\\Old_Young_Walking_Data\\"
# load the data
grf_data = np.load(data_dir + 'OA_no_missteps_grf.npy', mmap_mode='r')
muscle_data = np.load(data_dir + 'OA_no_missteps_muscle.npy', mmap_mode='r')

grf_labels = ['GRF_x', 'GRF_y', 'GRF_z']
muscle_labels = ['tibpost', 'tibant', 'edl', 'ehl', 'fdl', 'fhl', 'perbrev', 'perlong', 'achilles']

grf_dict = {0: 'GRF_x', 1: 'GRF_y', 2: 'GRF_z'}
muscle_dict = {0: 'tibpost', 1: 'tibant', 2: 'edl', 3: 'ehl', 4: 'fdl', 5: 'fhl', 6: 'perbrev', 7: 'perlong', 8: 'achilles'}

In [3]:
data = np.concatenate((grf_data, muscle_data), axis=2)
# print(data.shape)

np.random.seed(42)
np.random.shuffle(data)

In [4]:
num_samples = data.shape[0]
train_size = int(0.8 * num_samples)
val_size = int(0.1 * num_samples)

print(f"Number of samples: {num_samples}")
print(f"Train size: {train_size}")
print(f"Validation size: {val_size}")

Number of samples: 766
Train size: 612
Validation size: 76


In [5]:
train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data
print(f"Train data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Test data shape: {test_data.shape}")

Train data shape: (612, 100, 12)
Validation data shape: (76, 100, 12)
Test data shape: (766, 100, 12)


In [6]:
X_train, y_train = train_data[:, :, :3], train_data[:, :, 3:]
X_val, y_val = val_data[:, :, :3], val_data[:, :, 3:]
X_test, y_test = test_data[:, :, :3], test_data[:, :, 3:]

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (612, 100, 3)
y_train shape: (612, 100, 9)
X_val shape: (76, 100, 3)
y_val shape: (76, 100, 9)
X_test shape: (766, 100, 3)
y_test shape: (766, 100, 9)


In [7]:
np.savez(data_dir + 'no_missteps_train_data.npz', X_train=X_train, y_train=y_train)
np.savez(data_dir + 'no_missteps_val_data.npz', X_val=X_val, y_val=y_val)
np.savez(data_dir + 'no_missteps_test_data.npz', X_test=X_test, y_test=y_test)